## Preprocessing

### Estrazione frasi da file conllu
Nel caso in cui dovessimo partire da i file conllu: estraiamo un campione di frasi a partire dai file conllu di Wikipedia italiana elaborata. 

In [5]:
import os
import re
import pandas as pd
from tqdm import tqdm

In [2]:
def get_id(line):
    sent_id = re.sub(r'\D', '', line)
    return sent_id

def get_text(line):
    sent = line[9:].rstrip('\n')
    return sent

def get_sentences(file_conllu):
    sentences = []
    ids = []
    with tqdm(total=sum(1 for _ in open(file_conllu, 'r', encoding='utf-8')), desc=f'Progresso estrazione frasi del file {file_conllu}') as pbar:
        for line in open(file_conllu, 'r', encoding='utf-8'):
            pbar.update(1)  # Aggiorna la barra di avanzamento
            if line.startswith("# sent_id"):
                current_id = get_id(line)  
                ids.append(current_id)
            elif line.startswith("# text"):
                current_sent = get_text(line)
                sentences.append(current_sent)
    return ids, sentences

In [3]:
#nel caso in cui dovessimo itereare sui diversi file nella dir
# ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"
# ds_files = []
# for file_name in os.listdir(ds_directory):
#     file_path = os.path.join(ds_directory, file_name)
#     ds_files.append(file_path)  #ottengo il nome dei diversi file all'interno della directory
# print(ds_files)

'ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"\nds_files = []\nfor file_name in os.listdir(ds_directory):\n    file_path = os.path.join(ds_directory, file_name)\n    ds_files.append(file_path)  #ottengo il nome dei diversi file all\'interno della directory\nprint(ds_files)'

In [4]:
# data_df = pd.DataFrame(columns=["id", "text"])
# data_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file"""

'data_df = pd.DataFrame(columns=["id", "text"])\ndata_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file'

### Caricamento dati

Nel caso in cui disponessimo già di un dataset si salvano i dati semplicemente in un DataFrame.

In [7]:
data_df = pd.read_csv("data/csv/more_frasi.csv", encoding="utf-8")

In [8]:
data_df.tail()

,sent_id,doc,text,gulpease
19995,294985,5,"Le sue scaramucce con i media, specialmente co...",57
19996,108578,12,La mancanza di ogni suo scritto rende problema...,34
19997,204204,3,Più fonti affermano che adottarono il Cristian...,38
19998,1559901,11,L'idea di una grazia salvifica è esclusa e l'e...,50
19999,1027778,6,Sulla calotta della statua sono presenti dei f...,44


In [9]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sent_id   20000 non-null  int64 
 1   doc       20000 non-null  int64 
 2   text      20000 non-null  object
 3   gulpease  20000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 625.1+ KB


In [13]:
INIZIO = 0
fine = len(data_df)
samplino = data_df.iloc[INIZIO:fine] 

In [14]:
samplino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sent_id   20000 non-null  int64 
 1   doc       20000 non-null  int64 
 2   text      20000 non-null  object
 3   gulpease  20000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 625.1+ KB


In [16]:
data_df.iloc[19999]

sent_id                                               1027778
doc                                                         6
text        Sulla calotta della statua sono presenti dei f...
gulpease                                                   44
Name: 19999, dtype: object

In [17]:
samplino.iloc[19999]

sent_id                                               1027778
doc                                                         6
text        Sulla calotta della statua sono presenti dei f...
gulpease                                                   44
Name: 19999, dtype: object

### Estrazione degli indici di leggibilità per ogni frase

Utilizziamo l'API _read-it_ per calcolare gli indici di complessità delle frasi. Si passano all'API come un singolo documento di x frasi, per velocizzare il processo. 

In [1]:
import requests
import sys
import json

In [2]:

SERVER_PATH = "http://api.italianlp.it"
#SERVER_PATH = "http://itanlp-gpu.ilc.cnr.it:13000"

def load_document(text, i, x):
    try:
        r = requests.post(SERVER_PATH + '/documents/',           # carica il documento nel database del server
                        data={'text': text,                    # durante il caricamento viene eseguita un'analisi linguistica necessaria per calcolare la leggibilità
                            'lang': 'IT',
                            'extra_tasks': ["readability"]     # chiede al server di calcolare anche la leggibilità del docuemnto
                    })
        r.raise_for_status()  
        doc_id = r.json()['id']                           # id del documento nel database del server, che serve per richiedere i risultati delle analisi     
        return doc_id
    except requests.RequestException as e:
        print(f"Errore nel caricamento dei dati: {e}\n Iterazione: {i+INIZIO} - {(i+INIZIO)+x}: ")
        with open('frasi_saltate_val.txt', 'a', encoding='utf-8') as frasi_saltate:
            frasi_saltate.write(f"{i+INIZIO} - {(i+INIZIO)+x} - Documento: \n{text}\n")
        return None
    

def get_doc_score(doc_id, i, x):
    try:
        r = requests.get(SERVER_PATH + '/documents/details/%s' % doc_id)  #richiesta all'API per ottenere i risultati 
        r.raise_for_status()  
        result = r.json()   
        return result
    except requests.RequestException as e:
        print(f"Errore nel recupero dei risultati: {e}\n Iterazione: {i+INIZIO} - {(i+INIZIO)+x}: ")
        return None




In [12]:
#facciamo una prova con un testo di poche frasi
text = "Il cielo notturno era illuminato da milioni di stelle scintillanti. La vita è un viaggio, non una destinazione. La musica è il linguaggio universale dell'umanità. Le piccole cose nella vita spesso portano le più grandi gioie. Non esiste un ascensore per il successo; devi prendere le scale. Ogni giorno è una nuova opportunità per essere felici. I fiori non si preoccupano di come sbocceranno; semplicemente lo fanno. La risata è il miglior antidoto contro lo stress. La gentilezza è un linguaggio che i sordi possono sentire e i ciechi possono vedere. Nella semplicità risiede la vera bellezza."


doc_id = load_document(text, None, None)
r_score = get_doc_score(doc_id, None, None)

#vediamo com'è strutturato l'oggetto che viene restituito dall'API
print(r_score)

In [3]:
#qui iteriamo sui risultati ottenuti per un singolo documento
def get_sen_scores(doc_scores):
    all_sent_rscore = []
    all_sent_text = []
    for sent_results in doc_scores['sentences']['data']:
        all_sent_rscore.append(sent_results['readability_score_all'])
        all_sent_text.append(sent_results['raw_text'])
    return all_sent_text, all_sent_rscore

#si passa all'api un doc di 1000 frasi per volta.
def readability_extraction(sen_list, x): #x è il numero di frasi che si vogliono calcolare per iterazione 
    for i in tqdm(range(0, len(sen_list), x), desc=f"Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a {x} frasi)"):
        doc = "\n".join(sen_list[i:i+x])
        doc_id = load_document(doc, i, x)
        r_scores = get_doc_score(doc_id, i, x)
        with open(f"C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/readit_scores/more_frasi/{i+INIZIO}-{(i+INIZIO)+x}.json", "w") as f:
            json.dump(r_scores, f)          #si salva per ogni "doc" di 1000 frasi un file json con i risultati




'\ndef get_final_data(result_dict):\n    text_list = []\n    score_list = []\n    for doc_scores in result_dict:\n        text, score = get_sen_scores(doc_scores)\n        text_list = text_list + text\n        score_list = score_list + score\n    return text_list, score_list\n'

In [20]:
all_results = readability_extraction(samplino["text"].tolist(), 1000)  #si passa all'estrazione

Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a 1000 frasi): 100%|██████████| 20/20 [54:35<00:00, 163.78s/it]


### Estrazione da JSON

In [6]:
#codice per estrarre score dai json
#iteriamo sui diversi file json nella directory
json_dir = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/readit_scores/more_frasi"
ds_files = []
for file_name in os.listdir(json_dir):
    file_path = os.path.join(json_dir, file_name)
    ds_files.append(file_path) #ottengo il nome dei diversi file all'interno della directory

In [7]:
#si leggono i file json per ottenere lista di frasi e lista di punteggi read-it
all_sent = []
all_scores = []
for file_name in tqdm(ds_files, desc="Processing files"):
    with open(file_name, "r", encoding="utf-8") as json_file:
        result = json.load(json_file)
        #print(result["sentences"]["count"])
    if result != None:
        doc_sen, doc_scores = get_sen_scores(result)
        all_sent = all_sent + doc_sen
        all_scores = all_scores + doc_scores
    else: 
        print(f'File vuoto: {file_name}\n')
        

Processing files:   0%|          | 0/20 [00:00<?, ?it/s]

Processing files: 100%|██████████| 20/20 [00:02<00:00,  9.29it/s]


In [8]:
len(all_scores)

20000

In [9]:
len(all_sent)

20000

### Creazione e valutazione del dataset finale

Si ottiene una struttura dataframe con _id_, _testo e _leggibilità_ per ogni frase. Dopodiché valutiamo le caratteristiche dei dati ottenuti

In [11]:
#si crea un dataframe con tutte le informazioni che ci interessano
data_set = pd.DataFrame(columns=["id", "text", "readability"])
data_set["text"] = all_sent
data_set["readability"] = all_scores
data_set["id"] = range(1, len(data_set) + 1)
data_set.head()

,id,text,readability
0,1,"Hanno bisogno delle stesse qualità di energia,...",49.710611
1,2,"Mostacciano è divisa in tre zone che, in origi...",82.657379
2,3,"C'erano 84 nuclei familiari di cui il 35,7% av...",77.539188
3,4,"Gestisce inoltre un'importante biblioteca di ""...",89.894156
4,5,"Il suo poliedro duale è l'icosaedro, anch'esso...",67.113349


In [12]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           20000 non-null  int64  
 1   text         20000 non-null  object 
 2   readability  19817 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 468.9+ KB


In [150]:
valori_nulli = data_set[data_set["readability"].isnull()]
valori_nulli #vediamo che le frasi troppo corte non restituiscono un valore di leggibilità. 

,id,text,readability
39,40,Ferro di cavallo di Lander,NaN
63,64,Ltd.,NaN
454,455,Guillaume Emmanuel Guignard de Saint-Priest,NaN
539,540,De Agostini - ISBN 8841820055,NaN
547,548,Joaquin Fernandez de Portocarrero Mendoza,NaN
...,...,...,...
999363,999364,Squadra sudafricana di Coppa Davis,NaN
999650,999651,Modalità di gioco.,NaN
999685,999686,Scutt.,NaN
999716,999717,Funzionamento.,NaN


Le frasi passate all'API sono sottoposte a un ulteriore splitting. Per questo motivo alcune frasi vengono suddivise in più parti rispetto a quelle originali. Tutte quelle frasi vengono rimosse

In [32]:
#si rimuovono le frasi che non matchano con quelle originali
clean_ds = data_set[data_set['text'].isin(data_df['text'])] 

In [38]:
clean_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19817 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           19817 non-null  int64  
 1   text         19817 non-null  object 
 2   readability  19817 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 619.3+ KB


In [13]:
#si eliminano anche i restanti elementi che hanno leggibilità NaN
clean_ds = data_set.dropna()

In [14]:
clean_ds["id"] = range(1, len(clean_ds)+1) #id vengono riassegnati 

C:\Users\bergo\AppData\Local\Temp\ipykernel_17772\1643316225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_ds["id"] = range(1, len(clean_ds)+1)


In [16]:
clean_ds.tail()

,id,text,readability
19995,19813,Il distretto è formato da 2 città e 28 comuni,1.127732
19996,19814,Dopo essersi aggiudicato le simpatie del re di...,85.201601
19997,19815,"Senza queste necessità, la dimenticherebbero.",54.502640
19998,19816,un organismo comune avrebbe dovuto assicurare ...,60.542846
19999,19817,Il giovane Murakami era però più attratto dai ...,36.954094


In [39]:
clean_ds.to_csv("more_frasi_ws.csv", index=False) #una volta pronto salviamo il file